In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# RAM workable at 25% rescaled
rescaled_size = (216,384)
input_shape = (None, 216, 384, 3)
flatten_shape = (None, 82944, 3)
threshold_for_results = 0.5
# RAM exhausted at 50% rescaled
#rescaled_size = (540,960)
#input_shape = (None, 540, 960, 3)

In [6]:
# To ensure results can be repeated, the same seed value should be used for all testing
seed_value= 2019

from numpy.random import seed
seed(seed_value)
# Additional seed value required to be set for tensorflow backend
from tensorflow import set_random_seed
set_random_seed(seed_value)

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Seed values re-set to 2019.")

Seed values re-set to 2019.


In [7]:
# Imports to read images
from matplotlib import pyplot as plt
#from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from math import sqrt
from skimage.color import rgb2gray
import glob
from skimage.io import imread
print("Import Done")

Import Done


In [0]:
# Imports for model building from tutorial site
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [9]:
# Imports from previous model building
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
#!pip install numpy==1.16.1
import numpy as np

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

print("finish import")

finish import


In [0]:
# Imports for image downscaling because it takes too much RAM
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

In [0]:
# Test code for rescaling
#image1 = image.load_img('/content/drive/My Drive/Colab Notebooks/3) CNN Test 1 Data/non-face (' +'123'+ ')' + '.png', grayscale=False, target_size = rescaled_size)
#image1

In [12]:
# Part 1/4 of data input
# Read in the training images
# There are only 324 pictures to train and validate
# Separate 24 picture kept for testing
# non-face (252)
# 'non-face (' + i + ')' + '.png'

train_image = []
for i in tqdm(range(1,253)):
    stringIndex = str(i)
    img = image.load_img('/content/drive/My Drive/Colab Notebooks/3) CNN Test 1 Data/non-face (' + stringIndex + ')' + '.png', grayscale=False, target_size = rescaled_size)
    img = image.img_to_array(img)
    img = img/255 
    train_image.append(img)

print(len(train_image))
#X = np.array(train_image) # wait for train face image below

100%|██████████| 252/252 [00:13<00:00, 17.61it/s]

252


In [13]:
# Part 2/4 of data input
# Read in the training images
# There are only 324 pictures to train and validate
# Separate 24 picture kept for testing
# face (72)
# 'face (' + i + ')' + '.png'

for i in tqdm(range(1,73)):
    stringIndex = str(i)
    img = image.load_img('/content/drive/My Drive/Colab Notebooks/3) CNN Test 1 Data/face (' + stringIndex + ')' + '.png', grayscale=False, target_size = rescaled_size)
    img = image.img_to_array(img)
    img = img/255 
    train_image.append(img)

print(len(train_image))
X = np.array(train_image)

100%|██████████| 72/72 [00:03<00:00, 18.39it/s]


324


In [14]:
# Part 3/4 of data input
# Read in the test images
# Out of 24 test pictures, this half is non-face objects
# test-empty (12)
# 'test-empty (' + i + ')' + '.png'

train_image = [] # Remove the training data
for i in tqdm(range(1,13)):
    stringIndex = str(i)
    img = image.load_img('/content/drive/My Drive/Colab Notebooks/3) CNN Test 1 Data/test-empty (' + stringIndex + ')' + '.png', grayscale=False, target_size = rescaled_size)
    img = image.img_to_array(img)
    img = img/255 
    train_image.append(img)

print(len(train_image))
#X_test = np.array(train_image)

100%|██████████| 12/12 [00:00<00:00, 27.91it/s]

12


In [15]:
# Part 4/4 of data input
# Read in the test images
# Out of 24 test pictures, this half is face objects
# test-face (12)
# 'test-face (' + i + ')' + '.png'

for i in tqdm(range(1,13)):
    stringIndex = str(i)
    img = image.load_img('/content/drive/My Drive/Colab Notebooks/3) CNN Test 1 Data/test-face (' + stringIndex + ')' + '.png', grayscale=False, target_size = rescaled_size)
    img = image.img_to_array(img)
    img = img/255 
    train_image.append(img)

print(len(train_image))
X_test = np.array(train_image)

100%|██████████| 12/12 [00:00<00:00, 30.40it/s]

24


In [16]:
train_image = [] # Clear data

print(X.shape)
print(X_test.shape)

(324, 216, 384, 3)
(24, 216, 384, 3)


In [17]:
# Create y values for training data
y = [0] * 252
a = [1] * 72

for i in a:
  y.append(a[i])

y = np.asarray(y)
print(type(y))
print(len(y)) # Total length should be 324 training data
#y = to_categorical(y) # one hot encoding? but binary does not need, not multiclass

print(y[251]) # Last of non-face
print(y[252]) # First of face

<class 'numpy.ndarray'>
324
0
1


In [18]:
# Create y values for test data
y_test = [0] * 12
a = [1] * 12

for i in a:
  y_test.append(a[i])

y_test = np.asarray(y_test)
print(type(y_test))
print(len(y_test)) # Total length should be 324 training data
#y = to_categorical(y) # one hot encoding? but binary does not need, not multiclass

print(y_test[11]) # Last of non-face
print(y_test[12]) # First of face

<class 'numpy.ndarray'>
24
0
1


In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=2019, test_size=0.2)
X_train_flatten = X_train.reshape(259,82944,3)
X_test_flatten = X_test.reshape(24,82944,3)
X_val_flatten = X_val.reshape(65,82944,3)

In [20]:
np.random.shuffle(X_train)
np.random.shuffle(X_train_flatten)
np.random.shuffle(X_val)
np.random.shuffle(X_val_flatten)
print("Shuffled Training Data")

Shuffled Training Data


In [21]:
print(X_train.shape)
print(X_train_flatten.shape)
print(X_test_flatten.shape)
print(X_val.shape)
print(X_val_flatten.shape)
print(len(y_train))
print(len(y_val))

(259, 216, 384, 3)
(259, 82944, 3)
(24, 82944, 3)
(65, 216, 384, 3)
(65, 82944, 3)
259
65


In [0]:
###########################################################################################################################
#------------------------------------------------------------Model Building-----------------------------------------------#
###########################################################################################################################

In [23]:
# User defined parameters
batch_size_user = 2
epoch_user = 3

print("User defined parameters set")
print(batch_size_user)
print(epoch_user)

User defined parameters set
2
3


In [24]:
## create the model
model = 0
model = Sequential()
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# build for different input shape
model.build(input_shape) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)

print(model.summary())





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 248832)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62208250  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 62,208,501
Trainable params: 62,208,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# To ensure results can be repeated, the same seed value should be used for all testing
seed_value= 2019

from numpy.random import seed
seed(seed_value)
# Additional seed value required to be set for tensorflow backend
from tensorflow import set_random_seed
set_random_seed(seed_value)

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Seed values re-set to 2019.")

Seed values re-set to 2019.


In [26]:
# Fit the model
model.fit(X_train, y_train, epochs=epoch_user, batch_size=batch_size_user, validation_data = (X_val, y_val),verbose=1)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 259 samples, validate on 65 samples
Epoch 1/3
259/259 [==============================] - 12s 48ms/step - loss: 3.5523 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 2/3
259/259 [==============================] - 11s 41ms/step - loss: 3.6095 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 3/3
259/259 [==============================] - 11s 41ms/step - loss: 3.6095 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846


In [27]:
# Test the model
y_pred = model.predict(X_test)
y_pred = y_pred.reshape(24)
print(y_pred.shape)
print(y_test.shape)

print(y_pred)

avg = np.average(y_pred)
print("Average value is: ", avg)

y_pred[y_pred<avg] = 0
y_pred[y_pred>=avg] = 1

print(y_pred)
print(y_test)
totalsum = 0
for i in range(y_test.size):
  if(y_pred[i] == y_test[i]):
    totalsum += 1

print(totalsum / y_test.size * 100, "%")


(24,)
(24,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Average value is:  0.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 %


In [0]:
###########################################################################################################################
#------------------------------------------------------------Model 2-----------------------------------------------#
###########################################################################################################################

In [28]:
# create the model
model2 = 0
model2 = Sequential()
#model2.add(Flatten())
#model2.add(GRU(units=64, input_shape=(125000000,1,1),return_sequences=True))
#model2.add(GRU(units=32, return_sequences=True))
#model2.add(GRU(units=16, return_sequences=True))
#model2.add(GRU(units=8, return_sequences=True))
model2.add(GRU(units=4, input_shape=(82944,3)))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# build for different input shape
model2.build(flatten_shape) # `input_shape` is the shape of the input data

print(model2.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 4)                 96        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
# To ensure results can be repeated, the same seed value should be used for all testing
seed_value= 2019

from numpy.random import seed
seed(seed_value)
# Additional seed value required to be set for tensorflow backend
from tensorflow import set_random_seed
set_random_seed(seed_value)

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Seed values re-set to 2019.")

Seed values re-set to 2019.


In [30]:
# Fit the model, using flatten X data
model2.fit(X_train_flatten, y_train, epochs=epoch_user, batch_size=batch_size_user, validation_data = (X_val_flatten, y_val),verbose=1)

Train on 259 samples, validate on 65 samples
Epoch 1/3


KeyboardInterrupt: ignored

In [0]:
# Test the model
y_pred2 = model2.predict(X_test_flatten)
y_pred2 = y_pred2.reshape(24)
print(y_pred2.shape)
print(y_test.shape)

print(y_pred2)

avg = np.average(y_pred2)
print("Average value is: ", avg)

y_pred2[y_pred2<avg] = 0
y_pred2[y_pred2>=avg] = 1

print(y_pred2)
print(y_test)
totalsum = 0
for i in range(y_test.size):
  if(y_pred2[i] == y_test[i]):
    totalsum += 1

print(totalsum / y_test.size * 100, "%")


(24,)
(24,)
[0.20313421 0.20934808 0.2688833  0.35604703 0.4071105  0.49826947
 0.20044506 0.2027174  0.46728623 0.47548032 0.46889377 0.46986598
 0.26180565 0.19449347 0.19394383 0.3768788  0.19559604 0.4869374
 0.50380504 0.50380504 0.50380504 0.47407904 0.50380504 0.50380504]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1.]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
70.83333333333334 %


In [0]:
###########################################################################################################################
#------------------------------------------------------------Model 3-----------------------------------------------#
###########################################################################################################################

In [31]:
# create the model
model3 = 0
model3 = Sequential()
model3.add(Conv1D(filters=16, kernel_size=5, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Flatten())
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# build for different input shape
model3.build(flatten_shape) # `input_shape` is the shape of the input data

print(model3.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 82944, 16)         256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 41472, 16)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 663552)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 663553    
Total params: 663,809
Trainable params: 663,809
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
# To ensure results can be repeated, the same seed value should be used for all testing
seed_value= 2019

from numpy.random import seed
seed(seed_value)
# Additional seed value required to be set for tensorflow backend
from tensorflow import set_random_seed
set_random_seed(seed_value)

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Seed values re-set to 2019.")

Seed values re-set to 2019.


In [33]:
# Fit the model, using flatten X data
model3.fit(X_train_flatten, y_train, epochs=epoch_user, batch_size=batch_size_user, validation_data = (X_val_flatten, y_val),verbose=1)

Train on 259 samples, validate on 65 samples
Epoch 1/3
259/259 [==============================] - 4s 15ms/step - loss: 3.6279 - acc: 0.7683 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 2/3
259/259 [==============================] - 3s 11ms/step - loss: 3.6095 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 3/3
259/259 [==============================] - 3s 11ms/step - loss: 3.6095 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846


In [34]:
# Test the model
y_pred3 = model3.predict(X_test_flatten)
y_pred3 = y_pred3.reshape(24)
print(y_pred3.shape)
print(y_test.shape)

print(y_pred3)

avg = np.average(y_pred3)
print("Average value is: ", avg)

y_pred3[y_pred3<=avg] = 0
y_pred3[y_pred3>avg] = 1

print(y_pred3)
print(y_test)
totalsum = 0
for i in range(y_test.size):
  if(y_pred3[i] == y_test[i]):
    totalsum += 1

print(totalsum / y_test.size * 100, "%")


(24,)
(24,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Average value is:  0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
50.0 %


In [0]:
###########################################################################################################################
#------------------------------------------------------------Model 4-----------------------------------------------#
###########################################################################################################################

In [36]:
# create the model
model4 = 0
model4 = Sequential()
model4.add(Conv2D(16, kernel_size=(5, 5),activation='relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Flatten())
model4.add(Dense(1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# build for different input shape
model4.build(input_shape) # `input_shape` is the shape of the input data

print(model4.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 212, 380, 16)      1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 106, 190, 16)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 322240)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 322241    
Total params: 323,457
Trainable params: 323,457
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
# To ensure results can be repeated, the same seed value should be used for all testing
seed_value= 2019

from numpy.random import seed
seed(seed_value)
# Additional seed value required to be set for tensorflow backend
from tensorflow import set_random_seed
set_random_seed(seed_value)

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Seed values re-set to 2019.")

Seed values re-set to 2019.


In [38]:
# Fit the model
model4.fit(X_train, y_train, epochs=epoch_user, batch_size=batch_size_user, validation_data = (X_val, y_val),verbose=1)

Train on 259 samples, validate on 65 samples
Epoch 1/3
259/259 [==============================] - 3s 10ms/step - loss: 3.5529 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 2/3
259/259 [==============================] - 2s 8ms/step - loss: 3.6095 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846
Epoch 3/3
259/259 [==============================] - 2s 8ms/step - loss: 3.6095 - acc: 0.7761 - val_loss: 3.4716 - val_acc: 0.7846


In [73]:
# Test the model
y_pred4 = model4.predict(X_test)
y_pred4= y_pred4.reshape(24)
print(y_pred4.shape)
print(y_test.shape)

print(y_pred4)

avg = np.average(y_pred4)
print("Average value is: ", avg)

y_pred4[y_pred4<=avg] = 0
y_pred4[y_pred4>avg] = 1

print(y_pred4)
print(y_test)
totalsum = 0
for i in range(y_test.size):
  if(y_pred4[i] == y_test[i]):
    totalsum += 1

print(totalsum / y_test.size * 100, "%")


FailedPreconditionError: ignored

In [0]:
###########################################################################################################################
#------------------------------------------------------------Model 5-----------------------------------------------#
###########################################################################################################################

In [97]:
# create the model
model5 = 0
model5 = Sequential()
model5.add(Conv2D(16, kernel_size=(5, 5),activation='relu'))
model5.add(Conv2D(8, (5, 5), activation='relu'))
model5.add(Conv2D(4, (5, 5), activation='relu'))
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(Flatten())
model5.add(Dense(1, activation='sigmoid'))
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# build for different input shape
model5.build(input_shape) # `input_shape` is the shape of the input data

print(model5.summary())

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 212, 380, 16)      1216      
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 208, 376, 8)       3208      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 204, 372, 4)       804       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 102, 186, 4)       0         
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 51, 93, 4)         0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 18972)             0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)               

In [98]:
# To ensure results can be repeated, the same seed value should be used for all testing
seed_value= 2019

from numpy.random import seed
seed(seed_value)
# Additional seed value required to be set for tensorflow backend
from tensorflow import set_random_seed
set_random_seed(seed_value)

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print("Seed values re-set to 2019.")

Seed values re-set to 2019.


In [99]:
# Fit the model
model5.fit(X_train, y_train, epochs=epoch_user, batch_size=batch_size_user, validation_data = (X_val, y_val),verbose=1)

Train on 259 samples, validate on 65 samples
Epoch 1/3
259/259 [==============================] - 5s 18ms/step - loss: 0.6833 - acc: 0.7568 - val_loss: 0.6178 - val_acc: 0.7846
Epoch 2/3
259/259 [==============================] - 3s 12ms/step - loss: 0.5578 - acc: 0.7761 - val_loss: 0.5237 - val_acc: 0.7846
Epoch 3/3
259/259 [==============================] - 3s 12ms/step - loss: 0.5388 - acc: 0.7761 - val_loss: 0.5291 - val_acc: 0.7846


In [100]:
# Test the model
y_pred5 = model5.predict(X_test)
y_pred5= y_pred5.reshape(24)
print(y_pred5.shape)
print(y_test.shape)

print(y_pred5)

avg = np.average(y_pred5)
print("Average value is: ", avg)

y_pred5[y_pred5<=avg] = 0
y_pred5[y_pred5>avg] = 1

print(y_pred5)
print(y_test)
totalsum = 0
for i in range(y_test.size):
  if(y_pred5[i] == y_test[i]):
    totalsum += 1

print(totalsum / y_test.size * 100, "%")


(24,)
(24,)
[0.13395968 0.14205521 0.14006689 0.1490475  0.1389358  0.1339513
 0.15017414 0.14548627 0.13809803 0.13555998 0.1361064  0.13291919
 0.1662134  0.16550967 0.1592584  0.1673224  0.16147569 0.16743457
 0.16799292 0.16642398 0.16372967 0.15803534 0.15980673 0.16090003]
Average value is:  0.15168597
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
100.0 %
